In [114]:
import json
import pickle
import random
from collections import defaultdict
from datetime import datetime
from itertools import chain
from math import log
from typing import Iterable

import keras
import numpy
import pandas
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from matplotlib import pyplot
from scipy import stats
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, StratifiedKFold
from statsmodels.formula import api as formula_api
from statsmodels.stats.multitest import multipletests

%matplotlib inline
pandas.set_option('display.max_columns', 500)

In [115]:
# Define cross-validation and evaluation strategies.

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = 'accuracy'

In [116]:
# Parse battle logs.

def parse_heroes(heroes: Iterable[dict], sign: int, result: dict):
    for hero in heroes:
        result[f'level_{hero["id"]}'] += sign * hero['level']
        result[f'color_{hero["id"]}'] += sign * hero['color']
        result[f'star_{hero["id"]}'] += sign * hero['star']


def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    parse_heroes(battle.get('attackers') or battle['player'], +1, result)
    parse_heroes(battle.get('defenders') or battle['enemies'], -1, result)
        
    return {'win': battle['win'], **result}

In [117]:
# Load the logs into a data frame.

lines = list(chain.from_iterable([
    open('battles.jsonl'),
    open('battles-lilia.jsonl'),
]))

numpy.random.seed(42)
battles = pandas.DataFrame([parse_battle(line) for line in lines])
battles.drop_duplicates(inplace=True)  # because the files may contain duplicate battles
# battles.dropna(axis=1, thresh=5, inplace=True)  # require some number of battles for each hero
battles.fillna(value=0.0, inplace=True)
print(f'Shape: {battles.shape}')
battles.head()

Shape: (364, 121)


,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_23,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_37,color_38,color_39,color_4,color_40,color_41,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_23,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_37,level_38,level_39,level_4,level_40,level_41,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_23,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_37,star_38,star_39,star_4,star_40,star_41,star_5,star_6,star_7,star_8,star_9,win
0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
1,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,False
2,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False


In [118]:
battles.describe()

,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_23,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_37,color_38,color_39,color_4,color_40,color_41,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_23,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_37,level_38,level_39,level_4,level_40,level_41,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_23,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_37,star_38,star_39,star_4,star_40,star_41,star_5,star_6,star_7,star_8,star_9
count,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000
mean,-0.508242,3.222527,1.557692,-1.222527,3.618132,-0.379121,-0.818681,0.076923,-0.450549,0.153846,-0.173077,1.483516,-1.436813,-0.167582,-0.197802,-0.065934,1.491758,-0.395604,-0.274725,-0.115385,-0.104396,-0.214286,-0.134615,0.019231,-0.335165,-0.082418,-0.156593,0.200549,-0.129121,-0.019231,0.291209,-0.189560,-2.450549,-0.085165,-0.010989,-0.123626,-0.755495,-0.513736,-0.846154,-0.340659,-3.145604,27.126374,12.200549,-7.760989,28.236264,-2.821429,-6.159341,0.763736,-3.604396,2.752747,-1.359890,10.975275,-11.263736,-1.321429,-1.585165,-0.469780,11.760989,-3.164835,-2.096154,-0.747253,-0.782967,-1.571429,-1.065934,0.151099,-2.785714,-0.747253,-1.189560,1.802198,-0.972527,-0.137363,2.502747,-1.519231,-19.178571,-0.802198,-0.129121,-1.000000,-4.755495,-3.788462,-6.925824,-2.068681,-0.159341,1.837912,0.725275,-0.483516,1.637363,-0.142857,-0.208791,0.041209,-0.175824,0.074176,-0.057692,0.796703,-0.480769,-0.065934,-0.060440,-0.035714,0.090659,-0.129121,-0.109890,-0.027473,-0.030220,-0.060440,-0.085165,0.005495,-0.115385,-0.043956,-0.057692,0.076923,-0.035714,-0.008242,0.129121,-0.054945,-0.947802,-0.002747,0.010989,-0.060440,-0.211538,0.071429,-0.315934,-0.162088
std,2.766872,3.933670,3.541096,3.577930,3.929096,1.943897,5.062157,0.978904,1.789247,3.906407,1.427877,3.954598,3.201905,1.245131,1.149630,0.656432,4.204476,1.643950,1.447452,1.598169,0.887677,1.314557,1.039713,0.366900,1.541984,0.883780,1.315383,2.108856,1.131725,0.366900,1.786953,1.255204,4.390106,0.850150,0.378325,0.964146,2.591169,4.813014,2.533187,1.415925,22.414901,32.345338,27.815625,29.237331,30.664257,15.486188,41.225937,8.331903,14.281992,31.935703,11.194235,32.0332

In [119]:
# Split the dataframe.

x = battles.drop(['win'], axis=1)
y = battles['win']

# Feature selection.

p_values = [
    stats.chi2_contingency(pandas.crosstab(x[column], y))[1]
    # stats.ttest_ind(x[column][y == False], x[column][y == True])[1]
    for column in x.columns
]
rejected, _, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')
print(f'Rejected: {rejected.sum()}')
print(f'Selected: {x.columns[rejected]}')

x2 = x.drop(columns=x.columns[~rejected])

Rejected: 16
Selected: Index(['color_12', 'color_15', 'color_18', 'color_2', 'color_7', 'color_9',
       'level_15', 'level_18', 'level_2', 'level_7', 'level_9', 'star_12',
       'star_15', 'star_18', 'star_7', 'star_9'],
      dtype='object')


In [120]:
# Train, adjust hyper-parameters and evaluate.

numpy.random.seed(42)

classifier = RandomForestClassifier(class_weight='balanced', n_jobs=5, random_state=42)
param_grid = {
    'n_estimators': list(range(1, 1000)),
    'max_features': list(range(1, x.shape[1] + 1)),
    'criterion': ['entropy', 'gini'],
}

%time grid_search = RandomizedSearchCV(classifier, param_grid, cv=cv, scoring=scoring, n_iter=20).fit(x, y)
best_estimator = grid_search.best_estimator_

print(f'Best score: {grid_search.best_score_}')
print(f'Best params: {grid_search.best_params_}')
print(f'Classes: {best_estimator.classes_}')

scores = cross_val_score(best_estimator, x, y, scoring=scoring, cv=cv)
print(f'CV score: {scores.mean()} (std: {scores.std()})')

CPU times: user 2min 37s, sys: 13.1 s, total: 2min 50s
Wall time: 1min 47s
Best score: 0.7802197802197802
Best params: {'n_estimators': 65, 'max_features': 13, 'criterion': 'gini'}
Classes: [False  True]
CV score: 0.7797832078653996 (std: 0.05670664990902467)


In [121]:
# Re-train the best model on the entire dataset.

best_estimator.fit(x, y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features=13,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=65, n_jobs=5,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [122]:
# Display the feature importances.

pandas.DataFrame({'Feature': x.columns, 'Importance': best_estimator.feature_importances_}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False) \
    .head(n=10)

,Importance
Feature,
level_2,0.058783
level_15,0.043454
level_7,0.040551
level_25,0.036335
level_4,0.035438
level_13,0.031688
level_12,0.030440
color_7,0.029581
level_18,0.028297


In [123]:
# Display prediction examples.

result = pandas.concat((
    pandas.Series(best_estimator.predict(x), index=battles.index, name='win_predicted'),
    pandas.Series(best_estimator.predict_proba(x)[:, 1], index=battles.index, name='win_probability'),
    x,
), axis=1)
result['win_probability'] = result['win_probability'].apply('{:.2f}'.format)
result.head(10)

,win_predicted,win_probability,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_23,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_37,color_38,color_39,color_4,color_40,color_41,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_23,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_37,level_38,level_39,level_4,level_40,level_41,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_23,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_37,star_38,star_39,star_4,star_40,star_41,star_5,star_6,star_7,star_8,star_9
0,False,0.14,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0
1,False,0.02,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0
2,False,0.09,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,0.02,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.06,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0
5,False,0.00,0.0,0.0,0.0,-3.0,0.0,0.0,-1.0,0.0,0.0,4.0,0.0,-1.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,-13.0,0.0,0.0,37.0,0.0,-9.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,-2.0,0.0,